In [1]:
import random
from math import pow
import os
import pickle
import pandas as pd
os.chdir('Resources/')

In [2]:
import random
from math import pow
import pandas as pd

def gcd(a, b):
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b
    else:
        return gcd(b, a % b)

def gen_key(q):
    key = random.randint(10**20, q)  # Ensure q is an integer
    while gcd(q, key) != 1:
        key = random.randint(10**20, q)
    return key

def power(a, b, c):
    x = 1
    y = a
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c
        y = (y * y) % c
        b = int(b / 2)
    return x % c

def generate_unique_keys():
    q = random.randint(10**20, 10**50)  # Large prime number
    g = random.randint(2, q)  # Generator value

    public_keys = []
    private_keys = []

    for _ in range(11):  # Generate for 11 attributes
        key = gen_key(q)  # Private key for receiver
        h = power(g, key, q)  # Public key component
        public_keys.append(h)
        private_keys.append(key)

    return public_keys, private_keys, q, g

public_keys, private_keys, q, g = generate_unique_keys()

with open('3_Public_Key.pkl', 'wb') as f_public:
    pickle.dump(public_keys, f_public)

with open('3_Private_Key.pkl', 'wb') as f_private:
    pickle.dump(private_keys, f_private)

In [3]:
import random
from math import pow
import pandas as pd
import pickle

def gcd(a, b):
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b
    else:
        return gcd(b, a % b)

def gen_key(q):
    key = random.randint(10**20, q)  # Ensure q is an integer
    while gcd(q, key) != 1:
        key = random.randint(10**20, q)
    return key

def power(a, b, c):
    x = 1
    y = a
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c
        y = (y * y) % c
        b = int(b / 2)
    return x % c

def encrypt(msg, q, h, g):
    en_msg = []
    k = gen_key(q)  # Private key for sender
    s = power(h, k, q)
    p = power(g, k, q)
    
    for i in range(0, len(msg)):
        en_msg.append(s * msg[i])  # Encrypt each element
    
    return en_msg, p

with open('3_Public_Key.pkl', 'rb') as f_public:
    public_keys = pickle.load(f_public)

df = pd.read_csv('1_Structured_Data.csv')

encrypted_data = []
p_list = []
encrypted_values_list = []

for i in range(11):
    attribute_values = df.iloc[:, i].tolist()

    encoded_values = pd.factorize(attribute_values)[0]

    encrypted_values, p = encrypt(encoded_values, q, public_keys[i], g)
    p_list.append(p)
    encrypted_values_list.append(encrypted_values)

    encrypted_data.append(encrypted_values)

    print(f"Attribute {i+1} - Public Key (h): {public_keys[i]}")
    print(f"Attribute {i+1} - Original Values:", attribute_values[:10])
    print(f"Attribute {i+1} - Encrypted Values:", encrypted_values[:10])

encrypted_df = pd.DataFrame(encrypted_data).T  # Transpose to match the structure
encrypted_df.columns = [f"Attribute_{i+1}" for i in range(11)]  # Label columns

encrypted_df.to_csv('4_Encrypted_Data.csv', index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_15664\3542813145.py:53: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  encoded_values = pd.factorize(attribute_values)[0]


Attribute 1 - Public Key (h): 6860436675823870168364175315382919673806091428661
Attribute 1 - Original Values: [40, 49, 37, 48, 54, 39, 45, 54, 37, 48]
Attribute 1 - Encrypted Values: [0, 342249060692439004219829778433976605319264172081, 684498121384878008439659556867953210638528344162, 1026747182077317012659489335301929815957792516243, 1368996242769756016879319113735906421277056688324, 1711245303462195021099148892169883026596320860405, 2053494364154634025318978670603859631915585032486, 1368996242769756016879319113735906421277056688324, 684498121384878008439659556867953210638528344162, 1026747182077317012659489335301929815957792516243]
Attribute 2 - Public Key (h): 9313761388355896382792078122541699255857671891111
Attribute 2 - Original Values: ['Male', 'Female', 'Male', 'Female', 'Male', 'Male', 'Female', 'Male', 'Male', 'Female']
Attribute 2 - Encrypted Values: [0, 11590806706456146075188807590941399623723532741891, 0, 11590806706456146075188807590941399623723532741891, 0, 0, 1159080

In [4]:
import random
from math import pow
import pandas as pd

def power(a, b, c):
    x = 1
    y = a
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c
        y = (y * y) % c
        b = int(b / 2)
    return x % c

# Decryption function
def decrypt(en_msg, p, key, q):
    dr_msg = []
    h = power(p, key, q)
    for i in range(0, len(en_msg)):
        dr_msg.append(int(en_msg[i] / h))  # Decrypt each element
    return dr_msg

# Read the dataset
df = pd.read_csv('1_Structured_Data.csv')

# Generate 11 unique public and private keys
public_keys, private_keys, q, g = generate_unique_keys()

# Encrypt and Decrypt each attribute
for i in range(11):
    # Extract the attribute values (convert to list)
    attribute_values = df.iloc[:, i].tolist()

    decrypted_values = decrypt(encrypted_values_list[i], p_list[i], private_keys[i], q)

    # Decode the decrypted values back into categorical values
    decoded_values = pd.factorize(attribute_values)[1][decrypted_values]

    # Print the results for the first 10 values
    print(f"Attribute {i+1} - Public Key (h): {public_keys[i]}")
    print(f"Attribute {i+1} - Private Key (key): {private_keys[i]}")
    print(f"Attribute {i+1} - Original Values:", attribute_values[:10])
    print(f"Attribute {i+1} - Encrypted Values:", encrypted_values[:10])
    print(f"Attribute {i+1} - Decrypted Values:", decoded_values[:10])



C:\Users\USER\AppData\Local\Temp\ipykernel_15664\3523058017.py:37: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  decoded_values = pd.factorize(attribute_values)[1][decrypted_values]


Attribute 1 - Public Key (h): 18359671740509617270274540820818728185497374150035
Attribute 1 - Private Key (key): 14922118976921841383178724268556238305216477938875
Attribute 1 - Original Values: [40, 49, 37, 48, 54, 39, 45, 54, 37, 48]
Attribute 1 - Encrypted Values: [0, 20373688072306344616245136781199848321440102458681, 0, 20373688072306344616245136781199848321440102458681, 0, 0, 0, 0, 20373688072306344616245136781199848321440102458681, 0]
Attribute 1 - Decrypted Values: [40 40 40 40 40 40 40 40 40 40]
Attribute 2 - Public Key (h): 14574676132404261579478454584465647325039595128503
Attribute 2 - Private Key (key): 12286164761803805034158934947444177665001212501613
Attribute 2 - Original Values: ['Male', 'Female', 'Male', 'Female', 'Male', 'Male', 'Female', 'Male', 'Male', 'Female']
Attribute 2 - Encrypted Values: [0, 20373688072306344616245136781199848321440102458681, 0, 20373688072306344616245136781199848321440102458681, 0, 0, 0, 0, 2037368807230634461624513678119984832144010245868

IndexError: index 7 is out of bounds for axis 0 with size 4

In [ ]:
import random
from math import pow
import pandas as pd

def gcd(a, b):
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b
    else:
        return gcd(b, a % b)

def gen_key(q):
    key = random.randint(10**20, q)  # Ensure q is an integer
    while gcd(q, key) != 1:
        key = random.randint(10**20, q)
    return key

def power(a, b, c):
    x = 1
    y = a
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c
        y = (y * y) % c
        b = int(b / 2)
    return x % c

# Encryption function
def encrypt(msg, q, h, g):
    en_msg = []
    k = gen_key(q)  # Private key for sender
    s = power(h, k, q)
    p = power(g, k, q)
    
    for i in range(0, len(msg)):
        en_msg.append(s * msg[i])  # Encrypt each element
    
    return en_msg, p

# Decryption function
def decrypt(en_msg, p, key, q):
    dr_msg = []
    h = power(p, key, q)
    for i in range(0, len(en_msg)):
        dr_msg.append(int(en_msg[i] / h))  # Decrypt each element
    return dr_msg

# Function to generate 11 unique public and private keys
def generate_unique_keys():
    q = random.randint(10**20, 10**50)  # Large prime number
    g = random.randint(2, q)  # Generator value

    public_keys = []
    private_keys = []

    for _ in range(11):  # Generate for 11 attributes
        key = gen_key(q)  # Private key for receiver
        h = power(g, key, q)  # Public key component
        public_keys.append(h)
        private_keys.append(key)

    return public_keys, private_keys, q, g

# Read the dataset
df = pd.read_csv('1_Structured_Data.csv')

# Generate 11 unique public and private keys
public_keys, private_keys, q, g = generate_unique_keys()

# First loop: Encryption of each attribute
encrypted_values_list = []
p_list = []

for i in range(11):
    # Extract the attribute values (convert to list)
    attribute_values = df.iloc[:, i].tolist()

    # Encoding categorical values as integers
    encoded_values = pd.factorize(attribute_values)[0]

    # Encrypt the values using the corresponding public key
    encrypted_values, p = encrypt(encoded_values, q, public_keys[i], g)

    # Store the encrypted values and p for future decryption
    encrypted_values_list.append(encrypted_values)
    p_list.append(p)

    # Print encryption results for the first 10 values
    print(f"Attribute {i+1} - Public Key (h): {public_keys[i]}")
    print(f"Attribute {i+1} - Encrypted Values:", encrypted_values[:10])

# Second loop: Decryption of each attribute
for i in range(11):
    # Decrypt the values using the corresponding private key
    decrypted_values = decrypt(encrypted_values_list[i], p_list[i], private_keys[i], q)

    # Decode the decrypted values back into categorical values
    attribute_values = df.iloc[:, i].tolist()
    decoded_values = pd.factorize(attribute_values)[1][decrypted_values]

    # Print decryption results for the first 10 values
    print(f"Attribute {i+1} - Private Key (key): {private_keys[i]}")
    print(f"Attribute {i+1} - Decrypted Values:", decoded_values[:10])


Attribute 1 - Private Key (key): 3459833900807099802856651230177101165578131091333
Attribute 1 - Decrypted Values: [40 49 37 48 54 39 45 54 37 48]
Attribute 2 - Private Key (key): 40892226745755403996936524019205790290360077424506
Attribute 2 - Decrypted Values: ['Male' 'Female' 'Male' 'Female' 'Male' 'Male' 'Female' 'Male' 'Male'
 'Female']
Attribute 3 - Private Key (key): 882551891569213125427430476777548307564420529159
Attribute 3 - Decrypted Values: ['Atypical Angina' 'Non-anginal Pain' 'Atypical Angina' 'Asymptomatic'
 'Non-anginal Pain' 'Non-anginal Pain' 'Atypical Angina' 'Atypical Angina'
 'Asymptomatic' 'Atypical Angina']
Attribute 4 - Private Key (key): 40496753361278360324727931984594747015504327918712
Attribute 4 - Decrypted Values: [140. 160. 130. 138. 150. 120. 130. 110. 140. 120.]
Attribute 5 - Private Key (key): 905472048179841186697998520154478672151255484356
Attribute 5 - Decrypted Values: [289. 180. 283. 214. 195. 339. 237. 208. 207. 284.]
Attribute 6 - Private Key (

C:\Users\USER\AppData\Local\Temp\ipykernel_15664\4213628479.py:80: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  encoded_values = pd.factorize(attribute_values)[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_15664\4213628479.py:100: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  decoded_values = pd.factorize(attribute_values)[1][decrypted_values]
